## Imports
---

In [ ]:
# Installation de Polyglot, une librairie qui supporte 136 langages pour l'analyse de sentiments.
!pip3 install polyglot
!pip3 install pyicu
!pip3 install pycld2
!pip3 install morfessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52580 sha256=c695129f9ff1b395aa60159f1abf8c8811b8992e6d7c827c8185a2deffbd4a56
  Stored in directory: /root/.cache/pip/wheels/77/4a/9d/5141018da475375d91dc1af07520b1f2b077579f2f55353afb
Successfully built polyglot
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 KB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.10.2-cp39-cp39-linux_x86_64.whl size=1834731 sha256=1318fde2e3c16ec894e478b4de6d0129c3c143a646a226d219dd60c8c986d0e5
  Stored 

In [ ]:
# Téléchargement du modèle sentiment2 pour tous les langages disponibles.
%%bash
polyglot download TASK:sentiment2

[polyglot_data] Downloading collection 'TASK:sentiment2'
[polyglot_data]    | 
[polyglot_data]    | Downloading package sentiment2.pam to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.it to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.hsb to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.sah to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.hi to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.fr to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.es to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.vi to
[polyglot_data]    |     /root/polyglot_data...
[polyglot_data]    | Downloading package sentiment2.ar to
[polyglot_da

In [ ]:
# Importation des modules Python
import os
import pandas as pd
import numpy as np
import unicodedata
import polyglot
from polyglot.text import Text

In [ ]:
# Importation de GoogleDrive
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/MOOC'

path = os.getcwd()
print(f"Le répertoire courant est : {path} \n")

Mounted at /content/drive
/content/drive/MyDrive/MOOC
Le répertoire courant est : /content/drive/MyDrive/MOOC 



In [ ]:
# Import d'une partie des colonnes de la Table Messages (voir les fichiers de génération des csv)
messages = pd.read_csv('messages.csv', index_col=False)

In [ ]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62137 entries, 0 to 62136
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  60573 non-null  object
 1   body      62137 non-null  object
dtypes: object(2)
memory usage: 971.0+ KB


In [ ]:
messages.dropna(inplace=True)

In [ ]:
# Calcul de la polarité qui prend en compte les situations d'échec (quand Polyglot ne reconnaît pas la langue, par exemple à cause d'un lien html).
def calc_polarity(body_text):
  text = Text(body_text)
  try:
    p = text.polarity
  except Exception:
    p = ''
  return p
messages['body'] = messages['body'].apply(calc_polarity)
messages.head()

,username,body
0,ambruleaux,0.777778
1,danielgv,0.0
2,EricBouchet,1.0
3,Virginiebio,1.0
4,edwigedk,


In [ ]:
messages['body'].replace('', np.nan, inplace=True)

In [ ]:
messages.dropna(inplace=True)

In [ ]:
# Regroupement des messages par utilisateur pour préparer la jointure avec la Table Users.
messages = messages.groupby(['username'], as_index=False).mean()
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15549 entries, 0 to 15548
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   username  15549 non-null  object 
 1   body      15549 non-null  float64
dtypes: float64(1), object(1)
memory usage: 243.1+ KB


In [ ]:
# Import d'une partie des colonnes de la Table Users
users = pd.read_csv('users.csv', index_col=False)

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56548 entries, 0 to 56547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   username            56548 non-null  object
 1   country             11428 non-null  object
 2   gender              47682 non-null  object
 3   level_of_education  12328 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [ ]:
# Left Join pour conserver les utilisateurs qui n'ont pas posté de messages (éventuellement utile pour l'analyse).
users_messages = pd.merge(users, messages, how='left', on='username')
users_messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56548 entries, 0 to 56547
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   username            56548 non-null  object 
 1   country             11428 non-null  object 
 2   gender              47682 non-null  object 
 3   level_of_education  12328 non-null  object 
 4   body                15549 non-null  float64
dtypes: float64(1), object(4)
memory usage: 2.6+ MB


In [ ]:
# Import d'une partie des colonnes de la Table Results
results = pd.read_csv('results.csv', index_col=False)

In [ ]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72153 entries, 0 to 72152
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   username   72153 non-null  object 
 1   course_id  72153 non-null  object 
 2   grade      72153 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.7+ MB


In [ ]:
# Inner Join pour conserver uniquement les utilisateurs avec des notes.
users_messages_results = pd.merge(users_messages, results, how='inner', on='username')
users_messages_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72153 entries, 0 to 72152
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   username            72153 non-null  object 
 1   country             16092 non-null  object 
 2   gender              72153 non-null  object 
 3   level_of_education  17566 non-null  object 
 4   body                5601 non-null   float64
 5   course_id           72153 non-null  object 
 6   grade               72153 non-null  float64
dtypes: float64(2), object(5)
memory usage: 4.4+ MB


In [ ]:
# Export du fichier csv qui servira pour l'analyse et la modélisation.
users_messages_results.to_csv('users_messages_results.csv', index=False)